## **Applio**
Ultimate voice cloning tool, meticulously optimized for unrivaled power, modularity, and user-friendly experience.

[Support](https://discord.gg/IAHispano) — [Discord Bot](https://discord.com/oauth2/authorize?client_id=1144714449563955302&permissions=1376674695271&scope=bot%20applications.commands) — [Find Voices](https://applio.org/models) — [GitHub](https://github.com/IAHispano/Applio)

<br>

### **Credits**
- Encryption method: [Hina](https://github.com/hinabl)
- Extra section: [Poopmaster](https://github.com/poiqazwsx)
- Main development: [Applio Team](https://github.com/IAHispano)

<br>



In [ ]:
# @title **Install Applio**

import codecs
import time
import os
import csv
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime

rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)

org_name = rot_47("Vkkgdj")
new_name = rot_47("kmjbmvh_hg")
uioawhd = rot_47(codecs.decode("pbbxa://oqbpcj.kwu/QIPqaxivw/Ixxtqw.oqb", "rot_13"))
uyadwa = codecs.decode("ncc.cl", "rot_13")
!git clone --depth 1 $uioawhd
!mv $org_name $new_name
%cd $new_name/

from IPython.display import clear_output, Javascript

clear_output()

E = Exception
B = print


def vidal_setup(ForceIn):
    L = "Kikpm.ovm.bu"
    K = "/content/"
    C = ForceIn

    def F():
        print("Installing pip packages...")
        subprocess.check_call(["pip", "install", "-r", "requirements.txt", "--quiet"])

    A = K + rot_47(L)
    G = K + rot_47(L)
    D = "/"
    if not os.path.exists(A):
        M = os.path.dirname(A)
        os.makedirs(M, exist_ok=True)
        print("No cached install found..")
        try:
            N = rot_47(
                codecs.decode(
                    "pbbxa://pcooqvonikm.kw/QIPqaxivw/Ixxtqw/zmawtdm/uiqv/Kwtij/Xvxcz.biz.oh",
                    "rot_13",
                )
            )
            subprocess.run(["wget", "-O", A, N])
            print("Download completed successfully!")
        except E as H:
            print(str(H))
            if os.path.exists(A):
                os.remove(A)
    if Path(A).exists():
        with tarfile.open(G, "r:gz") as I:
            for J in I.getmembers():
                O = os.path.join(D, J.name)
                try:
                    I.extract(J, D)
                except E as H:
                    print("Failed to extract a file")
                    C = True
            print(f"Extraction of {G} to {D} completed.")
        if os.path.exists(A):
            os.remove(A)
        if C:
            F()
            C = False
    else:
        F()


vidal_setup(False)
clear_output()
print("Finished installing requirements!")

In [ ]:
# @title **Start Applio**
import codecs
import threading
import urllib.request
import time
import ipywidgets as widgets
from IPython.display import display
!npm install -g localtunnel
new_name = rot_47("kmjbmvh_hg")
%cd "/content/program_ml"
uyadwa = codecs.decode("ncc.cl", "rot_13")
share_tunnel = True  # @param {type:"boolean"}
def start_applio():
    if share_tunnel:
        !python $uyadwa --listen
    else:
        !python $uyadwa --listen --share

%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir logs --bind_all

if "autobackups" not in globals():
    autobackups = False

if autobackups:
    thread = threading.Thread(target=backup_files)
    thread.start()

thread_applio = threading.Thread(target=start_applio)
thread_applio.start()

if share_tunnel:
    if not os.path.exists(codecs.decode("eip/cergenvarqf/cergenvarq_i2/s0T48x.cgu", "rot_13")):
        while not os.path.exists(codecs.decode("eip/cergenvarqf/cergenvarq_i2/s0T48x.cgu", "rot_13")):
            time.sleep(2)
        time.sleep(5)
    else:
        time.sleep(10)
    with open('url.txt', 'w') as file:
        file.write('')

    get_ipython().system_raw('lt --port 6969 >> url.txt 2>&1 &')

    time.sleep(4)

    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

    with open('url.txt', 'r') as file:
        tunnel_url = file.read()
        tunnel_url = tunnel_url.replace("your url is: ", "")

    print(f"Share Link: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m\n")

    password_endpoint_widget = widgets.Text(
    value=endpoint_ip,
    description='Password IP:',
    disabled=True
    )
    display(password_endpoint_widget)



while True:
    time.sleep(5)


### **Extra**
Enjoy extra options that can make it easier for you to use Applio


In [ ]:
# @title Mount Drive
# @markdown Mount the files from Google Drive to the Colab.
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Auto Backup
# @markdown When running it, it will be activated or deactivated previously to start up together with Applio.
LOGS_FOLDER = "/content/program_ml/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/ApplioBackup"

if "autobackups" not in globals():
    autobackups = False


def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated = False
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" directory
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")
                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                                print(
                                    f"Created backup folder: {backup_folderpath}",
                                    flush=True,
                                )
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(
                                    current_timestamp
                                )
                                if last_backup_timestamp is None:
                                    print(f"Backed up file: {filename}")
                                else:
                                    print(f"Updating backed up file: {filename}")
                                updated = True
                                fully_updated = False

            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        print(f"Deleted file: {filepath}")
                    del last_backup_timestamps[filepath]
                    updated = True
                    fully_updated = False

            if not updated and not fully_updated:
                print("Files are up to date.")
                fully_updated = True
                sleep_time = 15
            else:
                sleep_time = 0.1

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(sleep_time)

        except Exception as e:
            print(f"An error occurred: {str(e)}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")

In [ ]:
# @title Load a Backup
from google.colab import drive

# @markdown Put the exact name you put as your Model Name in Applio.
modelname = "My-Project"  # @param {type:"string"}
source_path = "/content/drive/MyDrive/ApplioBackup/" + modelname
destination_path = "/content/program_ml/logs/" + modelname
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/ApplioBackup/", backup_timestamps_file)
    time__ = os.path.join("/content/program_ml/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

In [ ]:
# @title Download all custom pretrains
import os
import urllib.request

%mkdir /content/program_ml/rvc/pretraineds/pretraineds_custom
pretrained_urls = [
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super32kG.pth",
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super32kD.pth",
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kG.pth",
    "https://huggingface.co/ORVC/Ov2Super/resolve/main/f0Ov2Super40kD.pth",
    "https://huggingface.co/MUSTAR/RIN_E3/resolve/main/RIN_E3_G.pth",
    "https://huggingface.co/MUSTAR/RIN_E3/resolve/main/RIN_E3_D.pth",
    "https://huggingface.co/SeoulStreamingStation/KLMv7s/resolve/main/KLMv7s_32k.zip",
]
output_directory = "/content/program_ml/rvc/pretraineds/pretraineds_custom"
for url in pretrained_urls:
    filename = os.path.join(output_directory, os.path.basename(url))
    urllib.request.urlretrieve(url, filename)

!unzip /content/program_ml/rvc/pretraineds/pretraineds_custom/KLMv7s_32k.zip -d /content/program_ml/rvc/pretraineds/pretraineds_custom
!rm /content/program_ml/rvc/pretraineds/pretraineds_custom/KLMv7s_32k.zip